<h2>Processamento de Dados</h2>

Agora vamos fazer algumas alterações em nosso dataset para prosseguirmos com nossa análise. Esse processo é chamado também de limpeza de dados.

### Sumário:

-   [1. Carregando bibliotecas e Dataset](#cell1)
-   [2. Valores NA](#cell2)
-   [3. Normalizando Dados](#cell3)
-   [4. Variável Indicadora](#cell4)

<h3>Carregando Biblioteca e Dataset</h3>

Vamos primeiramente carregar os dados e ferramentas que precisamos.

In [ ]:
# Carrega tidyverse

library(tidyverse)

A empresa IBM, mantém em seu site uma série de conjunto de dados. Nesse caso usaremos o <a href="https://developer.ibm.com/exchanges/data/all/airline/">Airline Dataset</a> que contém dados de mais de 200 milhões de voos domésticos feitos em território americano, coletados pelo Departamento de Estatísticas de Transporte dos Estados Unidos. Por questões didáticas usaremos a amostra desses dados e não o arquivo inteiro.

<ul>
    <li>data source: <a href="https://dax-cdn.cdn.appdomain.cloud/dax-airline/1.0.1/lax_to_jfk.tar.gz" target="_blank">https://dax-cdn.cdn.appdomain.cloud/dax-airline/1.0.1/lax_to_jfk.tar.gz</a></li>
    <li>data type: csv</li>
</ul>

In [ ]:
# url onde os dados estão localizados
# url <- "https://dax-cdn.cdn.appdomain.cloud/dax-airline/1.0.1/lax_to_jfk.tar.gz"

# download do arquivo
#download.file(url, destfile = "lax_to_jfk.tar.gz")

# extrair arquivo 
# remova tar = "internal se estiver rodando este programa localmente" 
untar("lax_to_jfk.tar.gz", tar = "internal")

# Lemos o arquivo csv => read_csv (Se for de um formato diferente use a função apropriada, verifique a documentação de readr)
companhias_aereas <- read_csv("lax_to_jfk/lax_to_jfk.csv",
                     col_types = cols(
                      'DivDistance' = col_number(),
                      'DivArrDelay' = col_number()
                      ))

<h3>Valores NA e Formatação</h3>

- `NA`: Quando um valor está faltando no dataset a linguagem R atribui o símbolo NA (Not Available) a ele.
- `NAN`: Valores impossíveis, como divisão por 0, são representados por NaN(not a number).

Podemos usar a função `is.na(x)`, para saber se um elemento x é NA.





In [ ]:
# Observamos que há uma série de dados faltando no dataset

head(companhias_aereas)

In [ ]:
# Contamos a quantidade de valores faltando no dataset
companhias_aereas %>%
  summarize(count = sum(is.na(CarrierDelay)))

Vamos usar `purrr::map()` para contar valores NA em cada coluna. 

`map()` aplica uma função ou fórmula num elemento

-   O símbolo `~` separa o lado esquerdo do direito da fórmula. Geralmente fórmulas são representadas assim: `y ~ x`, no exemplo a seguir de `map()`, usaremos uma função que só precisa do lado direito: sum(is.na(.))
-   O `.` representa cada coluna no dataset. 

Verifique a documentação `?map` e `?formula`, para mais informações.

In [ ]:
# Aplique a função sum(is.na(.)) para cada coluna.
# Em outras palavras, encontre a soma dos valores NA de cada coluna.

map(companhias_aereas, ~sum(is.na(.)))

In [ ]:
# Checamos as dimensões do dataset

dim(companhias_aereas)

Temos em nosso dataset as seguintes linhas de valores NA.

<ol>
    <li>"CarrierDelay": 2486 dados faltando</li>
    <li>"WeatherDelay": 2486 dados faltando</li>
    <li>"NASDelay": 2486 dados faltando</li>
    <li>"SecurityDelay" : 2486 dados faltando</li>
    <li>"LateAircraftDelay": 2486 dados faltando</li>
    <li>"DivDistance": 2855 dados faltando</li>
    <li>"DivArrDelay": 2855 dados faltando</li>
</ol>

Há algumas formas de lidar com valores NA. Nesse caso, faremos uma demonstração de como removê-los do dataset

In [ ]:
# Vamos armazenar em colunas_NA nosso dataset sem as colunas DivDistance e DivArrDelay
colunas_NA <- companhias_aereas %>% select(-DivDistance, -DivArrDelay)

# Dimensões do dataset
dim(colunas_NA)

# Imprime as primeiras linhas do dataset
head(colunas_NA)

Vamos verificar o tipo de dado usado em cada coluna, assim podemos nos certificar que eles estão no formato correto. Para isso usaremos algumas funções do pacote `dplyr`.


| dplyr function |        _all       |              _if |
| ------------- | :---------------: | ---------------: |
| mutate        |   `mutate_all()`  |    `mutate_if()` |
| filter        |   `filter_all()`  |    `filter_if()` |
| group_by      |  `group_by_all()` |  `group_by_if()` |
| summarize     | `summarize_all()` | `summarize_if()` |

O uso de `_all` nas funções significa a sua aplicação em todas as variáveis e o `_if` somente se determinada função é `TRUE`

In [ ]:
# Mostre as classes de cada variável no dataset companhias_aereas

companhias_aereas %>% 
    summarize_all(class) %>% 
    gather(variable, class)

In [ ]:
# Para futuras análises cada dado deve estar no devido formato
# Aqui pegamos a coluna FlightDate e a desmembramos nas colunas ano, mês e dia

datas_de_voo <- companhias_aereas %>% 
    separate(FlightDate, sep = "-", into = c("year", "month", "day"))

head(datas_de_voo)

In [ ]:
# Por último, observamos que as colunas ano, mês e dia estão representadas como caracteres
# Iremos modificá-los para inteiros
# A função abaixo pode ser traduzida como:
# "Modifique todos os valores nas colunas ano,mês e dia se eles forem caracteres"

datas_de_voo %>%
    select(year, month, day) %>%
    mutate_all(type.convert) %>%
    mutate_if(is.character, as.numeric)

<h3>Normalizando Dados</h3>

É o processo de trazer dados distintos para um mesmo "alcance". Suponha, por exemplo, que você queira comparar idades de pessoas com seus respectivos salários. Este tipo de dado (salário) invariávelmente teria mais peso no modelo de análise. A fim de solucionar esse problema vamos usar a distribuição normal que é dada pela seguinte fórmula:

$$x_{new} = \frac{x_{old} - \mu}{\sigma}$$

${\mu}$: É a média dos valores

${\sigma}$: É o desvio padrão

${X}$: É uma variável qualquer.

Geralmente o resultado é uma série de valores variando de 0 a 1.



In [ ]:
# Usaremos a coluna ArrDelay 
# No código abaixo mean() calcula a média de ArrDelay e sd() o seu desvio padrão.
# Os dados são armazenados em escala_Z 
# Por fim mostramos os 6 primeiros dados normalizados de ArrDelay na tela

escala_Z <- (companhias_aereas$ArrDelay - mean(companhias_aereas$ArrDelay)) / sd(companhias_aereas$ArrDelay)
head(escala_Z)

In [ ]:
head(companhias_aereas$ArrDelay)

<h3>Variável Indicadora</h3>

Variáveis Indicadoras são usadas para aplicarmos técnicas de análise em variáveis categóricas. Em nosso dataset, por exemplo temos siglas representando as companhias áreas, caso quissemos números em seu lugar (lembrando que eles serão somente representativos) podemos atribuir-lhes valores `dummy`

In [ ]:
companhias_aereas %>%
  mutate(dummy = 1) %>% # Coluna com um valor
  spread(
    key = Reporting_Airline, # Usamos a coluna Companhias Aereas e a espalhamos
    value = dummy,
    fill = 0) %>%
  slice(1:5)

Podemos visualizar através de variáveis indicadoras o número de voos reportados por uma companhias aérea.

In [ ]:
companhias_aereas %>% # Começamos com nosso dataset
   mutate(Reporting_Airline = factor(Reporting_Airline, #Separamos por fatores
                                     labels = c("AA", "AS", "DL", "UA", "B6", "PA (1)", "HP", "TW", "VX")))%>%
  ggplot(aes(Reporting_Airline)) + #aes() ajuda a criar gráficos estilizados
  stat_count(width = 0.5) + #ESpessura
  labs(x = "Number of data points in each airline") # Título do gráfico